# Using Keywords as a Factor

### (Started Jun 30, 2019)

Continuing from Notebook 01-FirstDataExploration I will begin analyzing the article titles for keywords that could have some interesting statistical properties.

## Information from the Previous Notebook

Now, it would be good to look through the keywords from the article titles and see if there are any keywords that are indicitive of positive or negative results.

The initial look will be very basic and intuitive as we are just take a first look.

Hypothesis:
* Words like positive and negative should be telling of which direction the stock will go. Also w.r.t. exploration there should be other words that may be helpful in determining the direction (and possibly the magnitude)

Collect all of the "important" words (using NLP practices) for a bag of words.

Seperate the events into classes, in this case will use quartiles (this can be changed later on). For each word that has "enough" frequency we want to get as much information as possible about the probability of a company ending in each quartile bracket.

## Table of Contents

1. ["Import and Settings"](#1)
2. ["Importing and Cleaning Data"](#2)
3. ["NLP Exploration"](#3)

## Imports and Settings
<a id="1"><a/>

In [154]:
# Imports

# Numerical Libraries
import numpy as np
from scipy.stats import skew, kurtosis
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Visual Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Local Package Libraries
import sys
sys.path.append("../..")

from src.data.make_dataset import clean_and_open_business_wire_data_01, get_raw_data
from src.features.general_helper_functions import GetPrices
from src.features.nlp_functions import *

In [2]:
# Settings

# Stop the warnings for chain in pandas...
pd.options.mode.chained_assignment = None

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

%matplotlib inline

## Importing and Cleaning the Data
<a id="2"></a>

In [3]:
_, watchlist_raw, stock_prices_raw = get_raw_data()

In [4]:
clinical_trial_df = clean_and_open_business_wire_data_01(None)
clinical_trial_df.time = pd.to_datetime(clinical_trial_df.time)

In [5]:
# Watchlist

# 0: Create a copy of the data
watchlist_df = watchlist_raw.copy()
print("Original size: ", watchlist_df.shape)

# 1: Get a list of the unique companies that have scraped article data
unique_companies = clinical_trial_df.ticker.unique()

# 2: Keep only the companies from the list
watchlist_df = watchlist_df.loc[watchlist_df.Ticker.isin(unique_companies)]
print("Final size: ", watchlist_df.shape)

watchlist_df.columns = ["ticker", "marketcap", "sector", "exchange"]

watchlist_df.head()


# Stock Prices

# 0: Make a copy of the stock prices here
prices_df = stock_prices_raw.copy()
print("Original size: ", prices_df.shape)

# 1: Reduce the new copy of prices to only the companies under our scope
prices_df = prices_df[unique_companies]
print("Final size: ", prices_df.shape)

# 2: Sort by date
prices_df.sort_index(inplace=True)

# 3: Ensure index is datetime object
prices_df.index = pd.to_datetime(prices_df.index)

prices_df.tail()

Original size:  (721, 4)
Final size:  (197, 4)
Original size:  (5402, 208)
Final size:  (5402, 197)


,ACAD,ACHC,ACOR,ADUS,AERI,AGIO,AIMT,AKCA,AKRX,ALDR,...,VRAY,VREX,WMGI,WVE,XENT,XLRN,XNCR,XON,YI,ZGNX
2019-06-17,25.42,34.00,7.68,72.56,33.29,50.66,20.54,24.39,4.34,12.45,...,8.95,28.21,31.31,27.43,24.45,38.36,32.55,7.60,6.75,40.30
2019-06-18,25.93,34.20,7.66,72.92,33.84,51.91,20.20,23.98,4.48,11.73,...,9.13,28.95,31.69,27.92,23.97,40.62,34.54,8.60,6.62,39.79
2019-06-19,26.62,34.04,7.54,75.89,32.89,51.42,20.07,22.80,4.65,11.55,...,9.11,29.26,31.88,26.93,24.16,39.92,34.65,8.55,6.62,40.37
2019-06-20,25.73,34.52,7.28,74.91,32.13,51.30,20.24,22.90,4.74,11.54,...,8.92,29.57,30.48,26.86,23.62,40.85,35.10,8.01,6.62,40.91
2019-06-21,26.00,34.72,7.49,74.63,30.91,50.78,20.00,24.26,4.78,11.60,...,8.72,29.21,30.09,26.18,23.15,40.18,35.16,7.57,6.49,40.62


### Filter for "Phase"

In [6]:
df_filtered_for_phase = clinical_trial_df.loc[clinical_trial_df.title.str.contains("phase")]
print("Filtered size: ", df_filtered_for_phase.shape)

Filtered size:  (226, 4)


### Filter the Stock Prices

In [184]:
# Get the stock prices for 30 days following each event
price_window = GetPrices(
    df_filtered_for_phase, 
    prices_df, 
    n_window=5
).add_prices_to_frame()

#.dropna(axis=0, inplace=True)
print(price_window.head())

def perc_return(value_matrix, i, j):
    return (value_matrix[j][i] / value_matrix[j][0]) - 1


return_values = np.array([
    np.array([
        perc_return(price_window.values, i, j) for i in range(1, price_window.values.shape[1])
    ]) for j in range(price_window.values.shape[0])
])


cols = ["R_{}".format(i) for i in range(return_values.shape[1])]

return_window = pd.DataFrame(return_values, index=price_window.index, columns=cols)

print(return_window.head())

      P_0    P_1    P_2    P_3    P_4
1     NaN    NaN    NaN    NaN    NaN
6   24.45  25.17  25.12  24.05  23.66
12  26.24  26.64  26.85  26.98  27.32
25  19.48  22.36  22.73  21.66  21.88
22  16.89  16.26  15.93  16.01  15.56
         R_0       R_1       R_2       R_3
1        NaN       NaN       NaN       NaN
6   0.029448  0.027403 -0.016360 -0.032311
12  0.015244  0.023247  0.028201  0.041159
25  0.147844  0.166838  0.111910  0.123203
22 -0.037300 -0.056838 -0.052102 -0.078745


In [187]:
# July 1, 2019 - Create a function that encapsulates the above code block
def compute_return_window(article_df, prices_df, n_window=30):
    
    # Get the stock prices for "n" days following each event
    price_window = GetPrices(
        article_df, 
        prices_df, 
        n_window=n_window
    ).add_prices_to_frame()


    return_values = np.array([
        np.array([
            perc_return(price_window.values, i, j) for i in range(1, price_window.values.shape[1])
        ]) for j in range(price_window.values.shape[0])
    ])


    cols = ["R_{}".format(i) for i in range(return_values.shape[1])]

    return pd.DataFrame(return_values, index=price_window.index, columns=cols)

return_window = compute_return_window(df_filtered_for_phase, prices_df, 30)

## NLP Feature Exploration
<a id="3"></a>

In this section we will clean up the title in df_filtered_for_phase data using:

* remove_white_spaces
* remove_non_alphanumeric
* remove_numbers
* remove_stop_words

In [160]:
# July 1, 2019
def clean_text(df, column_name):
    df[column_name] = df[column_name].apply(remove_white_spaces)
    df[column_name] = df[column_name].apply(remove_non_alphanumeric)
    df[column_name] = df[column_name].apply(remove_numbers)
    df[column_name] = df[column_name].apply(remove_stop_words)
    df[column_name] = df[column_name].apply(remove_stop_words)
    df[column_name] = df[column_name].apply(lemmatize_text)
    return df

In [163]:
articles = clean_text(df_filtered_for_phase, "title")

articles.head()

,time,title,ticker,article
1,2019-05-18,present phase clarity result pimavanserin adj...,ACAD,san diego--(business wire)--acadia pharmaceut...
6,2019-04-25,initiate phase clarity program pimavanserin a...,ACAD,san diego--(business wire)--acadia pharmaceut...
12,2019-03-27,positive phase study result trofinetide pedia...,ACAD,"san diego & cincinnati & melbourne, australia..."
25,2018-10-31,announce positive top line result phase clari...,ACAD,san diego--(business wire)--acadia pharmaceut...
22,2019-01-17,announce lancet neurology publication phase d...,ACOR,"ardsley, n.y.--(business wire)--acorda therap..."


Looking at the titles again I noticed that the company's name is also generally in the title. It would be best to remove that as the company will already be it's own feature implicitly.

In [9]:
company_names = watchlist_df.loc[watchlist_df.ticker.isin(unique_companies)].index.tolist()
articles.title = articles.title.apply(remove_company_name, args=(company_names,))

We will now need our set of unique keyword

In [10]:
combined_titles = " ".join(articles.title.values.tolist())

set_of_words = list(set(combined_titles.split(" ")))
set_of_words.remove("")

len(set_of_words)

857

We will also want to remove all "words" that have 2 or less characters:

In [11]:
set_of_words.remove("phase")

In [12]:
set_of_words = [word for word in set_of_words if len(word) > 2]

len(set_of_words)

827

Need a function that, given a data frame (or sub data frame) with the article titles, will give the frequency that the word occurs.

In [13]:
def calculate_word_frequency(word_list, df):
    d = {word: sum([1 if word in article else 0 for article in df.title.values])/df.shape[0] for word in word_list}
    
    return pd.Series(d, index = d.keys()).sort_values(ascending=False)

In [14]:
word_frequency = calculate_word_frequency(set_of_words, articles)
word_frequency.head(20)

trial        0.469027
clinical     0.314159
announce     0.309735
patient      0.305310
study        0.292035
results      0.269912
announces    0.256637
com          0.252212
patients     0.247788
pre          0.238938
ted          0.207965
line         0.199115
data         0.176991
treatment    0.172566
present      0.163717
positive     0.154867
met          0.150442
initiate     0.132743
cancer       0.123894
anal         0.119469
dtype: float64

So, what we are looking for are words that are in a *substantial* number of documents and provide *enough* classification information.

by substantial, in this case I will use an absolute cut-off of 5%

In [15]:
set_of_words = word_frequency.loc[word_frequency > 0.05].index
set_of_words

Index(['trial', 'clinical', 'announce', 'patient', 'study', 'results',
       'announces', 'com', 'patients', 'pre', 'ted', 'line', 'data',
       'treatment', 'present', 'positive', 'met', 'initiate', 'cancer', 'anal',
       'market', 'research', 'initiates', 'markets', 'first', 'report',
       'analysis', 'med', 'chi', 'pipeline', 'iii', 'cell', 'advanced',
       'annual', 'car', 'top', 'enroll', 'disease', 'end', 'meeting',
       'researchandmarkets', 'age', 'update', 'develop', 'society',
       'development', 'tumor', 'anti', 'presents', 'complete', 'reports',
       'enrollment', 'point', 'part', 'pivotal', 'carcinoma', 'iga', 'europe',
       'tumors'],
      dtype='object')

#### Positive and Negative Frequency Ratio

These numbers will give the frequency of each word in the positive return or negative return events divided by the total number of the positive or negative return events respectively.

I will build the function to take in the return number so I can build this part out to test on various days for optimality.

In [16]:
def split_return_window(ret_df, holding_period=0):
    '''returns the positive and negative dataframes, in that order.'''
    pos = ret_df.loc[ret_df["R_{}".format(holding_period)] > 0].index
    neg = ret_df.loc[ret_df["R_{}".format(holding_period)] <= 0].index
    
    return pos, neg

In [17]:
pos_ind, neg_ind = split_return_window(return_window, 0)

pos_articles = articles.loc[pos_ind]
neg_articles = articles.loc[neg_ind]

In [18]:
pos_word_freq = calculate_word_frequency(set_of_words, pos_articles)
neg_word_freq = calculate_word_frequency(set_of_words, neg_articles)

In [19]:
sent_df = pd.DataFrame([pos_word_freq, neg_word_freq], index = ["pos", "neg"]).T
sent_df.head(10)

,pos,neg
trial,0.468198,0.448622
patient,0.326855,0.325815
announce,0.325088,0.300752
study,0.298587,0.273183
clinical,0.291519,0.290727
announces,0.273852,0.260652
patients,0.266784,0.253133
com,0.257951,0.303258
results,0.234982,0.243108
line,0.226148,0.223058


Let's take a look at the difference between the columns. This would indicate a "lean" towards on direction or another.

In [20]:
sent_df["diff"] = sent_df.pos - sent_df.neg

In [21]:
sent_df.sort_values("diff", inplace=True, ascending=False)

top_10 = sent_df.iloc[:10]
bottom_10 = sent_df.iloc[:-10:-1]

In [22]:
top_10.T

,positive,top,initiate,initiates,first,study,announce,tumors,treatment,pivotal
pos,0.160777,0.093640,0.143110,0.123675,0.125442,0.298587,0.325088,0.077739,0.171378,0.070671
neg,0.122807,0.062657,0.112782,0.095238,0.097744,0.273183,0.300752,0.055138,0.150376,0.050125
diff,0.037970,0.030983,0.030328,0.028437,0.027697,0.025404,0.024336,0.022601,0.021002,0.020546


In [23]:
bottom_10.T

,present,pre,com,annual,data,end,ted,pipeline,disease
pos,0.120141,0.197880,0.257951,0.049470,0.151943,0.058304,0.183746,0.097173,0.060071
neg,0.182957,0.255639,0.303258,0.090226,0.187970,0.092732,0.218045,0.127820,0.090226
diff,-0.062816,-0.057759,-0.045308,-0.040756,-0.036026,-0.034428,-0.034300,-0.030646,-0.030155


Alright so, now we have a way to get the top and bottom scoring words.

Now would like to convert this notebook portion into a piece of production code. This will allow further investigation on what keywords are important as I can look to longer holding periods.

Remember, for now, I am simply looking for keywords as factors. They will likely be pushed in as contains_word = {True, False} later on.

In [24]:
def calculate_word_frequency(word_list, df):
    d = {word: sum([1 if word in article else 0 for article in df.title.values])/df.shape[0] for word in word_list}
    
    return pd.Series(d, index = d.keys()).sort_values(ascending=False)

def split_return_window(ret_df, holding_period=0):
    '''returns the positive and negative dataframes, in that order.'''
    pos = ret_df.loc[ret_df["R_{}".format(holding_period)] > 0].index
    neg = ret_df.loc[ret_df["R_{}".format(holding_period)] <= 0].index
    
    return pos, neg

def calculate_top_and_bottom_keywords(article_df, return_df, holding_period, cut_off=0.05):
    combined_titles = " ".join(articles.title.values.tolist())

    set_of_words = list(set(combined_titles.split(" ")))
    set_of_words.remove("")
    set_of_words.remove("phase")
    
    set_of_words = [word for word in set_of_words if len(word) > 2]

    word_frequency = calculate_word_frequency(set_of_words, articles)
    
    set_of_words = word_frequency.loc[word_frequency > cut_off].index
    
    pos_ind, neg_ind = split_return_window(return_df, holding_period)

    pos_articles = articles.loc[pos_ind]
    neg_articles = articles.loc[neg_ind]
    
    pos_word_freq = calculate_word_frequency(set_of_words, pos_articles)
    neg_word_freq = calculate_word_frequency(set_of_words, neg_articles)
    
    sent_df = pd.DataFrame([pos_word_freq, neg_word_freq], index = ["pos", "neg"]).T
    
    sent_df["diff"] = sent_df.pos - sent_df.neg
    
    sent_df.sort_values("diff", inplace=True, ascending=False)

    return sent_df["diff"].iloc[:10], sent_df["diff"].iloc[:-10:-1]

In [25]:
calculate_top_and_bottom_keywords(articles.title, return_window, 0, 0.05)

(positive     0.037970
 top          0.030983
 initiate     0.030328
 initiates    0.028437
 first        0.027697
 study        0.025404
 announce     0.024336
 tumors       0.022601
 treatment    0.021002
 pivotal      0.020546
 Name: diff, dtype: float64, present    -0.062816
 pre        -0.057759
 com        -0.045308
 annual     -0.040756
 data       -0.036026
 end        -0.034428
 ted        -0.034300
 pipeline   -0.030646
 disease    -0.030155
 Name: diff, dtype: float64)

Great! It works, will add to src and refactor there.

Further, we can create a dataframe that tracks which words are import through time.

### Thought Update

I would like to revise the work a little bit above. Since we are working with returns, it doesn't make much sense to use binary classification. What would be better is instead to use return metrics that are common in industry such as Sharpe Ratio, and classify the top words for that.

It would also be interested to look at the following:
1. Using Feature Importance from a Tree model
2. Using these concepts on the overall data to see if there are keywords that could be important for all articles.

## July 1, 2019

From here, I would like to start using industry metrics. There are several to choose from, so it would be great to implement a function that could take in the metric seperately. This will be done later, for now I will simply use something similar to a Sharpe Ratio.

$$ S = \frac{r_{portfolio} - r_{free}}{\sigma} $$

In this instance, we will assume the risk free return is zero and can be implemented later if need be.

To find the top and bottom *N* stocks, simply get order the words that had the best sharpe ratio.

**Steps**
1. Filter the set of words as before
2. For each word:
    * get the list of all event_ids of the articles containing a word
    * get the holding period's returns
    * calculated the metric

In [167]:
def calculate_word_frequency(word_list, df):
    d = {word: sum([1 if word in article else 0 for article in df.title.values])/df.shape[0] for word in word_list}
    
    return pd.Series(d, index = d.keys()).sort_values(ascending=False)


def clean_word_list(articles, cut_off):
    combined_titles = " ".join(articles.title.values.tolist())

    set_of_words = list(set(combined_titles.split(" ")))
    
    set_of_words = [word for word in set_of_words if len(word) > 3]

    word_frequency = calculate_word_frequency(set_of_words, articles)
    set_of_words = word_frequency.loc[word_frequency > cut_off].index
    return set_of_words

def naive_sharpe(series):
    expected_return = series.mean()
    dev = series.std()
    return expected_return / dev

def calculate_sharpe_ratio(article_df, return_df, holding_period, cut_off=0.05):
    set_of_words = clean_word_list(article_df, cut_off)
    
    # In case the user inserts a holding period longer than allowed
    holding_period = min([len(return_df.columns) - 1, holding_period])
    
    sharpe_dict = {}
    for word in set_of_words:
        event_id_list = article_df.loc[article_df.title.str.contains(word)].index.tolist()
        returns = return_df["R_{}".format(holding_period)].iloc[event_id_list]
        sharpe_dict[word] = naive_sharpe(returns)
        
    S = pd.Series(sharpe_dict, index=sharpe_dict.keys())
    S.sort_values(ascending=False, inplace=True)
    return S

res = calculate_sharpe_ratio(articles, return_window, 0, 0.05)

print(res.head(10))
print("")
print(res.tail(10))

develop        1.338537
development    1.248778
pivotal        0.735344
positive       0.606156
meet           0.555026
europe         0.459805
annual         0.442164
point          0.423565
first          0.408091
data           0.380608
dtype: float64

study                -0.056246
cancer               -0.067508
enroll               -0.098866
enrol                -0.098866
market               -0.107381
research             -0.112427
initiate             -0.177212
dose                 -0.247407
tumor                -0.335094
researchandmarkets   -0.338697
dtype: float64


### Full Test

Let's try the functionality on the entire article data frame instead of the filtered one!

In [87]:
clinical_trial_df = clean_text(clinical_trial_df, "title")
all_article_return_window = compute_return_window(clinical_trial_df, 
                                                  prices_df, 
                                                  n_window=30)

res_full = calculate_top_and_bottom_sharpe(clinical_trial_df, all_article_return_window, 30, 0.01)

Before cutoff 8155
After cutoff 280 



In [88]:
S[0].head(10)

file          0.100213
drug          0.099866
conference    0.099143
review        0.097758
view          0.097072
announces     0.096882
lion          0.096798
finan         0.096383
announce      0.096245
report        0.096028
dtype: float64

In [89]:
S[0].tail(10)

markets               0.089124
research              0.089087
search                0.089086
ology                 0.088967
care                  0.088208
rate                  0.087484
point                 0.084747
chan                  0.075342
hand                  0.074920
researchandmarkets    0.074810
dtype: float64

### Change Functionality

After thinking about the project I realized I would like to make a few changes. The reason is to get more flexbility with metrics. The output of the function will have the following columns:
* Expected Return
* Standard Deviation of Returns
* Skew of Returns
* Kurtosis of Returns
* Number of instances (Trades)

Then after that, it will be easier to look at various metrics using a seperate function.

In [183]:
def get_return_details_per_word(article_df, return_df, holding_period, cut_off=0.05):
    set_of_words = clean_word_list(article_df, cut_off)
    
    res_dict = {}
    for word in set_of_words:
        event_id_list = article_df.loc[article_df.title.str.contains(word)].index.tolist()
        #print(event_id_list)
        returns = return_df["R_{}".format(holding_period - 1)].iloc[event_id_list].dropna()
        res_dict[word] = [
            np.mean(returns), 
            np.std(returns), 
            skew(returns.values), 
            kurtosis(returns.values),
            returns.shape[0]/article_df.shape[0]
        ]
    
    cols = ["return", "dev", "skew", "kurt", "freq_occurance"]
    
    return pd.DataFrame(res_dict, index=cols)

def sharpe_ratio(row, holding_period, annual_risk_free_rate):
    scale_param = 252 / holding_period # This will be used to annualize the expected return 
                                       # and the deviation
    num = (scale_param * row["return"] - annual_risk_free_rate) 
    den = (np.sqrt(scale_param) * row["dev"])
    return num / den

holding_period = 30
risk_free_rate = 0.025

# In case the user inserts a holding period longer than allowed
holding_period = min([len(return_window.columns), holding_period])

res = get_return_details_per_word(articles, return_window, holding_period, 0.1).T    

res["sharpe_ratio"] = res.apply(sharpe_ratio, args=(holding_period, risk_free_rate), axis=1)

res.sort_values("sharpe_ratio", ascending=False)

,return,dev,skew,kurt,freq_occurance,sharpe_ratio
first,0.073607,0.147450,0.456285,-1.116461,0.075221,1.414034
positive,0.051755,0.134496,0.373000,-0.289800,0.115044,1.071286
result,0.059078,0.178998,2.363357,8.521557,0.207965,0.925543
line,0.052210,0.163599,0.225875,-0.954824,0.115044,0.888904
announce,0.047411,0.156897,0.333173,-0.739548,0.203540,0.836712
study,0.066944,0.227307,2.116217,5.379617,0.207965,0.830850
treat,0.049928,0.208747,2.251961,6.192323,0.146018,0.664431
clinical,0.051560,0.220976,2.059625,5.814816,0.238938,0.649435
data,0.022746,0.120187,0.336363,-0.190480,0.115044,0.487325
treatment,0.020971,0.139303,0.965044,-0.214014,0.123894,0.382884


## Summary

Alright, so I am not sure if I believe the results at the moment as they seem too good to be true for a first run.

**Interesting Results**
1. When filtered first for the word "phase" and then for the word "first" the Sharpe ratio turns out to be 1.4. This result seems to be very nice. The issue here is there is only a 7% occurrance out of the number of articles tagged with "phase". That equates to 17 instances. Is this statistically significant enough to trade?
2. When filtered first for the word "phase" and then for the word "positive" the Sharpe ratio turns out to be 1.1. There is 11.5% occurance which is a bit better than the word "first" occuring 26 times. Again, the question is, is this statistically significant?

**Short Comings**
A few major shortcoming derive from this piece of research, mainly in data collection.

1. The news articles are largely only from the last few years. There is clearly a survivor bias inherent in the data. 
2. Only a subset of the universe of bio-pharmaceutical stocks were used. Nothing outside of our Market Cap. range was used, so if a stock was successful, it would be out of bounds.
3. If a stock had delisted, it would no longer be in the watchlist and so that data is not reflected here.

**Next Steps**
1. The main way to overcome this is to go to a data vendor to collect delisted stocks over the past n-years then to scrape those articles. Further, would like to include all of the watchlist instead of just our subsection of the market cap.

2. Need to refactor this notebook into a unified code so it can be used on a larger scale once more data is collected.

3. Another thing that would be interested is to create a backtest that can help to look at how accurate the research is. Having 2 numbers to look at and correlate the statistics would help ensure that the functionality is working correctly.